In [1]:
import torch.nn.functional as F
import torchvision
from mmengine.model import BaseModel

class MMResNet50(BaseModel):
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.resnet50()

    def forward(self, imgs, labels=None, mode='tensor'):
        x = self.resnet(imgs)
        if mode == 'loss':
            return {'loss': F.cross_entropy(x, labels)}
        elif mode == 'predict':
            return x, labels
        elif mode == 'tensor':
            return x

/nfs/home/3002_hehui/.conda/envs/XMX/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from mmengine.analysis import get_model_complexity_info

input_shape = (3, 224, 224)
model = MMResNet50()
analysis_results = get_model_complexity_info(model, input_shape)

11/14 13:59:13 - mmengine - WARNING - Unsupported operator aten::add_ encountered 69 time(s)
11/14 13:59:13 - mmengine - WARNING - Unsupported operator aten::max_pool2d encountered 1 time(s)
11/14 13:59:13 - mmengine - WARNING - The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
data_preprocessor
11/14 13:59:18 - mmengine - WARNING - Unsupported operator aten::batch_norm encountered 53 time(s)
11/14 13:59:18 - mmengine - WARNING - Unsupported operator aten::adaptive_avg_pool2d encountered 1 time(s)


In [3]:
print(analysis_results['out_table'])


+------------------------+----------------------+------------+--------------+
| module                 | #parameters or shape | #flops     | #activations |
+------------------------+----------------------+------------+--------------+
| resnet                 | 25.557M              | 4.145G     | 11.115M      |
|  conv1                 |  9.408K              |  0.118G    |  0.803M      |
|   conv1.weight         |   (64, 3, 7, 7)      |            |              |
|  bn1                   |  0.128K              |  4.014M    |  0           |
|   bn1.weight           |   (64,)              |            |              |
|   bn1.bias             |   (64,)              |            |              |
|  layer1                |  0.216M              |  0.69G     |  4.415M      |
|   layer1.0             |   75.008K            |   0.241G   |   2.007M     |
|    layer1.0.conv1      |    4.096K            |    12.845M |    0.201M    |
|    layer1.0.bn1        |    0.128K            |    1.004M  | 

In [3]:
print(analysis_results['out_table'])


+------------------------+----------------------+------------+--------------+
| module                 | #parameters or shape | #flops     | #activations |
+------------------------+----------------------+------------+--------------+
| resnet                 | 25.557M              | 4.145G     | 11.115M      |
|  conv1                 |  9.408K              |  0.118G    |  0.803M      |
|   conv1.weight         |   (64, 3, 7, 7)      |            |              |
|  bn1                   |  0.128K              |  4.014M    |  0           |
|   bn1.weight           |   (64,)              |            |              |
|   bn1.bias             |   (64,)              |            |              |
|  layer1                |  0.216M              |  0.69G     |  4.415M      |
|   layer1.0             |   75.008K            |   0.241G   |   2.007M     |
|    layer1.0.conv1      |    4.096K            |    12.845M |    0.201M    |
|    layer1.0.bn1        |    0.128K            |    1.004M  | 

In [6]:
# 读取配置文件
from mmengine.config import Config
cfg = Config.fromfile('../configs/ins/mask2former_eva02_tiny_nwpu.py')
print(cfg.model)

{'type': 'Mask2Former', 'data_preprocessor': {'type': 'mmdet.DetDataPreprocessor', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'bgr_to_rgb': True, 'pad_size_divisor': 32, 'pad_mask': True, 'mask_pad_value': 0}, 'backbone': {'type': ViTEVA02, 'arch': 'tiny', 'img_size': (1024, 1024), 'patch_size': 14, 'final_norm': False, 'out_type': 'featmap', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmpretrain/v1.0/eva02/eva02-tiny-p14_pre_in21k_20230505-d703e7b1.pth', 'prefix': 'backbone.'}}, 'panoptic_head': {'type': 'Mask2FormerHead', 'in_channels': [192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192], 'strides': [4, 8, 16, 32], 'feat_channels': 256, 'out_channels': 256, 'num_things_classes': 10, 'num_stuff_classes': 0, 'num_queries': 60, 'num_transformer_feat_level': 32, 'pixel_decoder': {'type': 'MSDeformAttnPixelDecoder', 'num_

In [17]:
from mmengine.config import Config

cfg = Config(
    dict(
        model=dict(
                embed_dims=192,
                num_layers=12,
                num_heads=3,
                feedforward_channels=int(192 * 4 * 2 / 3)
        )
    )
)

In [19]:
cfg.model

{'embed_dims': 192,
 'num_layers': 12,
 'num_heads': 3,
 'feedforward_channels': 512}

In [23]:
# get Parameters and FLOPs from config
from mmpretrain.models import ViTEVA02
from mmengine.analysis import get_model_complexity_info
                

input_shape = (3, 1024, 1024)
model = ViTEVA02(cfg.model)
analysis_results = get_model_complexity_info(model, input_shape)
print(analysis_results['out_table'])


+------------------------+----------------------+------------+--------------+
| module                 | #parameters or shape | #flops     | #activations |
+------------------------+----------------------+------------+--------------+
| resnet                 | 25.557M              | 86.578G    | 0.232G       |
|  conv1                 |  9.408K              |  2.466G    |  16.777M     |
|   conv1.weight         |   (64, 3, 7, 7)      |            |              |
|  bn1                   |  0.128K              |  83.886M   |  0           |
|   bn1.weight           |   (64,)              |            |              |
|   bn1.bias             |   (64,)              |            |              |
|  layer1                |  0.216M              |  14.42G    |  92.275M     |
|   layer1.0             |   75.008K            |   5.042G   |   41.943M    |
|    layer1.0.conv1      |    4.096K            |    0.268G  |    4.194M    |
|    layer1.0.bn1        |    0.128K            |    20.972M | 